## ICIS Claim Data Processing Tutorial
### Step-by-Step Guide for ICIS Claim Data Processing

**Author**: Seokhoon Joo  

### Table of Contents
* [1. Setup and Data Loading](#1.Setup-and-Data-Loading)
    * [1.1 Import Required Libraries](#1.1-Import-Required-Libraries)
    * [1.2 Load ICIS Claim Data](#1.2-Load-Claim-Data)
    * [1.3 Load Main Disease Classification Data](#1.3-Load-Main-Disease-Classification-Data)
    * [1.4 Initialize ICIS Processor](#1.4-Initialize-ICIS-Processor)
* [2. Step-by-Step Processing](#2.Step-by-Step-Processing)
    * [2.1 Data Validation](#2.1-Data-Validation)
    * [2.2 Data Cleansing](#2.2-Data-Cleansing)
    * [2.3 Data Preparation](#2.3-Data-Preparation)
    * [2.4 Data Calculations](#2.4-Data-Calculations)
    * [2.5 Merge Calculated Data](#2.5-Merge-Calculated-Data)
* [3. Complete Pipeline Processing](#3.Complete-Pipeline-Processing)
    * [3.1 Pipeline Execution](#3.1-Pipeline-Execution)
    * [3.2 Results Validation](#3.2-Results-Validation)
    * [Appendix: Error Handling](#Appendix:-Error-Handling)

### 1. Setup and Data Loading

#### 1.1 Import Required Libraries

In [1]:
import pandas as pd
from underwriter.icis import ICIS

#### 1.2 Load ICIS Claim Data

In [2]:
claim = pd.read_csv('data/claim.csv')
print("Initial claim data:")
print("Shape:", claim.shape)
print("\nColumns:", claim.columns.tolist())
print("\nFirst few rows:")
display(claim.head())

Initial claim data:
Shape: (11, 13)

Columns: ['id', 'kcd0', 'kcd1', 'kcd2', 'kcd3', 'kcd4', 'clm_date', 'hos_sdate', 'hos_edate', 'hos_day', 'hos_cnt', 'out_cnt', 'sur_cnt']

First few rows:


,id,kcd0,kcd1,kcd2,kcd3,kcd4,clm_date,hos_sdate,hos_edate,hos_day,hos_cnt,out_cnt,sur_cnt
0,100000001,NaN,M51,NaN,C44,NaN,20150102,20150102.0,20150108.0,4,1,0,0
1,100000001,NaN,M51,NaN,C44,NaN,20150102,20150102.0,20150108.0,4,1,0,0
2,100000001,S33,G551,NaN,NaN,NaN,20150102,20150102.0,20150105.0,0,2,2,0
3,100000001,M512,NaN,NaN,NaN,NaN,20200901,NaN,NaN,0,0,0,2
4,100000001,S33,M54,M513,NaN,NaN,20220802,20220802.0,20220806.0,5,0,0,0


#### 1.3 Load Main Disease Classification Data

In [3]:
main = pd.read_csv('data/main.csv')
print("\nMain disease classification data:")
print("Shape:", main.shape)
print("\nColumns:", main.columns.tolist())
print("\nFirst few rows:")
display(main.head())


Main disease classification data:
Shape: (10, 3)

Columns: ['kcd', 'kcd_main', 'sub_chk']

First few rows:


,kcd,kcd_main,sub_chk
0,C73,C73,1
1,D12,D12,1
2,K20,K20,0
3,M51,M51,1
4,M512,M51,1


#### 1.4 Initialize ICIS Processor

In [4]:
icis = ICIS(claim=claim, main=main)

### 2. Step-by-Step Processing

#### 2.1 Data Validation

In [5]:
print("\n2.1 Data Validation")
print("-----------------")
try:
    icis.validate_columns()
    print("✓ Column validation successful")
except ValueError as e:
    print(f"✗ Validation error: {e}")


2.1 Data Validation
-----------------
✓ Column validation successful


#### 2.2 Data Cleansing

In [6]:
print("\n2.2 Data Cleansing")
print("----------------")

print("• Initial claim data shape:", icis.claim.shape)
display(icis.claim.head())

print("\n1) Removing duplicates...")
icis.drop_duplicates()
print("• Shape after deduplication:", icis.claim.shape)
display(icis.claim.head())

print("\n2) Forward filling KCD codes...")
icis.fill_kcd_forward()
print("• Shape after forward fill:", icis.filled.shape)
display(icis.filled.head())

print("\n3) Filtering by claim date...")
icis.filter_by_clm_date()
print("• Shape after date filtering:", icis.filled.shape)
display(icis.filled.head())


2.2 Data Cleansing
----------------
• Initial claim data shape: (11, 13)


,id,kcd0,kcd1,kcd2,kcd3,kcd4,clm_date,hos_sdate,hos_edate,hos_day,hos_cnt,out_cnt,sur_cnt
0,100000001,NaN,M51,NaN,C44,NaN,2015-01-02,2015-01-02,2015-01-08,4,1,0,0
1,100000001,NaN,M51,NaN,C44,NaN,2015-01-02,2015-01-02,2015-01-08,4,1,0,0
2,100000001,S33,G551,NaN,NaN,NaN,2015-01-02,2015-01-02,2015-01-05,0,2,2,0
3,100000001,M512,NaN,NaN,NaN,NaN,2020-09-01,NaT,NaT,0,0,0,2
4,100000001,S33,M54,M513,NaN,NaN,2022-08-02,2022-08-02,2022-08-06,5,0,0,0



1) Removing duplicates...
• Shape after deduplication: (10, 13)


,id,kcd0,kcd1,kcd2,kcd3,kcd4,clm_date,hos_sdate,hos_edate,hos_day,hos_cnt,out_cnt,sur_cnt
0,100000001,NaN,M51,NaN,C44,NaN,2015-01-02,2015-01-02,2015-01-08,4,1,0,0
1,100000001,S33,G551,NaN,NaN,NaN,2015-01-02,2015-01-02,2015-01-05,0,2,2,0
2,100000001,M512,NaN,NaN,NaN,NaN,2020-09-01,NaT,NaT,0,0,0,2
3,100000001,S33,M54,M513,NaN,NaN,2022-08-02,2022-08-02,2022-08-06,5,0,0,0
4,100000001,S33,M54,D12,K20,M514,2022-08-02,2022-08-04,2022-08-04,1,0,0,0



2) Forward filling KCD codes...
• Shape after forward fill: (10, 13)


,id,kcd0,kcd1,kcd2,kcd3,kcd4,clm_date,hos_sdate,hos_edate,hos_day,hos_cnt,out_cnt,sur_cnt
0,100000001,M51,C44,NaN,NaN,NaN,2015-01-02,2015-01-02,2015-01-08,4,1,0,0
1,100000001,S33,G551,NaN,NaN,NaN,2015-01-02,2015-01-02,2015-01-05,0,2,2,0
2,100000001,M512,NaN,NaN,NaN,NaN,2020-09-01,NaT,NaT,0,0,0,2
3,100000001,S33,M54,M513,NaN,NaN,2022-08-02,2022-08-02,2022-08-06,5,0,0,0
4,100000001,S33,M54,D12,K20,M514,2022-08-02,2022-08-04,2022-08-04,1,0,0,0



3) Filtering by claim date...
• Shape after date filtering: (8, 13)


,id,kcd0,kcd1,kcd2,kcd3,kcd4,clm_date,hos_sdate,hos_edate,hos_day,hos_cnt,out_cnt,sur_cnt
0,100000001,M512,NaN,NaN,NaN,NaN,2020-09-01,NaT,NaT,0,0,0,2
1,100000001,S33,M54,M513,NaN,NaN,2022-08-02,2022-08-02,2022-08-06,5,0,0,0
2,100000001,S33,M54,D12,K20,M514,2022-08-02,2022-08-04,2022-08-04,1,0,0,0
3,100000001,M51,NaN,NaN,NaN,NaN,2022-08-02,2022-08-04,2022-08-07,4,0,0,0
4,100000001,M51,W00,NaN,NaN,NaN,2024-02-05,2024-02-08,2024-02-09,2,0,0,1


#### 2.3 Data Preparation

In [7]:
print("\n2.3 Data Preparation")
print("------------------")

print("1) Setting medical care types...")
icis.set_type()
print("• Data with medical care types:")
display(icis.filled[['id', 'clm_date', 'type']].head())

print("\n2) Modifying hospital end dates...")
icis.set_hos_edate_mod()
print("• Data with modified hospital end dates:")
display(icis.filled[['id', 'hos_edate', 'hos_edate_mod']].head())

print("\n3) Converting to long format...")
icis.melt()
print("• Melted data shape:", icis.melted.shape)
display(icis.melted.head())

print("\n4) Processing KCD information...")
icis.set_sub_kcd()
icis.merge_main_info()
icis.filter_sub_kcd()
print("• Shape after KCD processing:", icis.melted.shape)
display(icis.melted.head())


2.3 Data Preparation
------------------
1) Setting medical care types...
• Data with medical care types:


,id,clm_date,type
0,100000001,2020-09-01,sur
1,100000001,2022-08-02,hos
2,100000001,2022-08-02,hos
3,100000001,2022-08-02,hos
4,100000001,2024-02-05,hos/sur



2) Modifying hospital end dates...
• Data with modified hospital end dates:


,id,hos_edate,hos_edate_mod
0,100000001,NaT,NaT
1,100000001,2022-08-06,2022-08-06
2,100000001,2022-08-04,2022-08-04
3,100000001,2022-08-07,2022-08-07
4,100000001,2024-02-09,2024-02-09



3) Converting to long format...
• Melted data shape: (18, 12)


,id,clm_date,hos_sdate,hos_edate,hos_edate_mod,hos_day,hos_cnt,out_cnt,sur_cnt,type,kcd_ord,kcd
0,100000001,2020-09-01,NaT,NaT,NaT,0,0,0,2,sur,0,M512
1,100000001,2022-08-02,2022-08-02,2022-08-06,2022-08-06,5,0,0,0,hos,0,S33
2,100000001,2022-08-02,2022-08-04,2022-08-04,2022-08-04,1,0,0,0,hos,0,S33
3,100000001,2022-08-02,2022-08-04,2022-08-07,2022-08-07,4,0,0,0,hos,0,M51
4,100000001,2024-02-05,2024-02-08,2024-02-09,2024-02-09,2,0,0,1,hos/sur,0,M51



4) Processing KCD information...
• Shape after KCD processing: (16, 15)


,id,clm_date,hos_sdate,hos_edate,hos_edate_mod,hos_day,hos_cnt,out_cnt,sur_cnt,type,kcd_ord,kcd,sub_kcd,kcd_main,sub_chk
0,100000001,2020-09-01,NaT,NaT,NaT,0,0,0,2,sur,0,M512,0,M51,1
1,100000001,2022-08-02,2022-08-02,2022-08-06,2022-08-06,5,0,0,0,hos,0,S33,0,S33,1
2,100000001,2022-08-02,2022-08-04,2022-08-04,2022-08-04,1,0,0,0,hos,0,S33,0,S33,1
3,100000001,2022-08-02,2022-08-04,2022-08-07,2022-08-07,4,0,0,0,hos,0,M51,0,M51,1
4,100000001,2024-02-05,2024-02-08,2024-02-09,2024-02-09,2,0,0,1,hos/sur,0,M51,0,M51,1


#### 2.4 Data Calculations

In [8]:
print("\n2.4 Data Calculations")
print("------------------")

print("1) Setting date ranges...")
icis.set_date_range()

print("\n2) Calculating hospitalization days...")
icis.calc_hos_day()
print("• Hospitalized data shape:", icis.hospitalized.shape)
display(icis.hospitalized.head())

print("\n3) Calculating surgery counts...")
icis.calc_sur_cnt()
print("• Surgery data shape:", icis.underwent.shape)
display(icis.underwent.head())

print("\n4) Calculating elapsed days...")
icis.calc_elp_day()
print("• Elapsed days data shape:", icis.elapsed.shape)
display(icis.elapsed.head())


2.4 Data Calculations
------------------
1) Setting date ranges...

2) Calculating hospitalization days...
• Hospitalized data shape: (4, 3)


,id,kcd_main,hos_day
0,100000001,D12,1
1,100000001,M51,8
2,100000001,M54,5
3,100000001,S33,5



3) Calculating surgery counts...
• Surgery data shape: (1, 3)


,id,kcd_main,sur_cnt
0,100000001,M51,2



4) Calculating elapsed days...
• Elapsed days data shape: (5, 4)


,id,kcd_main,elp_day_si,elp_day_std
0,100000001,C73,NaN,264.0
1,100000001,D12,900.0,900.0
2,100000001,M51,346.0,264.0
3,100000001,M54,898.0,898.0
4,100000001,S33,898.0,898.0


#### 2.5 Merge Calculated Data

In [9]:
print("\n2.5 Final Merge")
print("-------------")
step_result = icis.merge_calculated()
print("• Final result shape:", step_result.shape)
print("• Final columns:", step_result.columns.tolist())
display(step_result.head())


2.5 Final Merge
-------------
• Final result shape: (5, 6)
• Final columns: ['id', 'kcd_main', 'hos_day', 'sur_cnt', 'elp_day_si', 'elp_day_std']


,id,kcd_main,hos_day,sur_cnt,elp_day_si,elp_day_std
0,100000001,C73,0.0,0.0,NaN,264.0
1,100000001,D12,1.0,0.0,900.0,900.0
2,100000001,M51,8.0,2.0,346.0,264.0
3,100000001,M54,5.0,0.0,898.0,898.0
4,100000001,S33,5.0,0.0,898.0,898.0


### 3. Complete Pipeline Processing

#### 3.1 Pipeline Execution

In [10]:
print("\n3.1 Pipeline Execution")
print("--------------------")

# Initialize new ICIS instance
icis_pipeline = ICIS(claim=claim, main=main)

# Process ICIS claim data using complete pipeline
print("Processing ICIS claim data using icis.process()...")
pipeline_result = icis_pipeline.process()
print("\n✓ Processing completed successfully!")
print("• Final result shape:", pipeline_result.shape)


3.1 Pipeline Execution
--------------------
Processing ICIS claim data using icis.process()...

✓ Processing completed successfully!
• Final result shape: (5, 6)


#### 3.2 Results Comparison

In [11]:
print("\n3.2 Results Validation")
print("--------------------")
# Compare results
print("\nResults Comparison:")
print("• Step-by-step shape:", step_result.shape)
print("• Pipeline shape:", pipeline_result.shape)

are_equal = step_result.equals(pipeline_result)
print(f"\n✓ Results are identical: {are_equal}")

if not are_equal:
    print("\nDifferences in columns:")
    print(set(step_result.columns) ^ set(pipeline_result.columns))


3.2 Results Validation
--------------------

Results Comparison:
• Step-by-step shape: (5, 6)
• Pipeline shape: (5, 6)

✓ Results are identical: True


#### Appendix: Error Handling

In [12]:
print("\nAppendix: Error Handling")
print("----------------")
# Example of error handling with invalid data
print("Testing error handling with invalid input...")

try:
    # Create invalid data for testing
    invalid_claim = claim.drop(columns=['id'])
    invalid_icis = ICIS(claim=invalid_claim, main=main)
    invalid_result = invalid_icis.process()
except ValueError as e:
    print(f"\n✓ Validation error caught successfully: {e}")
except RuntimeError as e:
    print(f"\n✓ Processing error caught successfully: {e}")
except Exception as e:
    print(f"\n✓ Unexpected error caught successfully: {e}")


Appendix: Error Handling
----------------
Testing error handling with invalid input...

✓ Validation error caught successfully: Missing required columns in claim DataFrame: ['id']
Required columns: ['id', 'kcd0', 'kcd1', 'kcd2', 'kcd3', 'kcd4', 'clm_date', 'hos_sdate', 'hos_edate', 'hos_day', 'hos_cnt', 'out_cnt', 'sur_cnt']
